In [1]:
import pandas as pd
import re
import pandas as pd 
import numpy as np 

# if we do not add engine='python', it will throw out a Unicode error
df = pd.read_csv('training.1600000.processed.noemoticon.csv', header=None, sep=',', engine='python')

sentiments = df.iloc[:, 0]
text = df.iloc[:, 5]
sentiments = np.array(sentiments)
text = np.array(text)

data = {'sentiment': sentiments, 'text': text}

df = pd.DataFrame(data)

In [2]:
def remove_mentions(data):
    return re.sub('@[\w]*', ' ', data)

df['text_cleaned']= df['text'].apply(lambda x: remove_mentions(x))

def remove_URLs(data):
    return re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',data)

df['text_cleaned']= df['text_cleaned'].apply(lambda x: remove_URLs(x))
df.head()


,sentiment,text,text_cleaned
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...","- Awww, that's a bummer. You shoulda got ..."
1,0,is upset that he can't update his Facebook by ...,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...,I dived many times for the ball. Managed to ...
3,0,my whole body feels itchy and like its on fire,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all....","no, it's not behaving at all. i'm mad. why a..."


In [3]:
df['text_cleaned'] = df['text_cleaned'].str.replace('[^a-zA-Z#]', ' ')

df.head(10)

<ipython-input-3-c4f8ac4d1c42>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['text_cleaned'] = df['text_cleaned'].str.replace('[^a-zA-Z#]', ' ')


,sentiment,text,text_cleaned
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",Awww that s a bummer You shoulda got ...
1,0,is upset that he can't update his Facebook by ...,is upset that he can t update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...,I dived many times for the ball Managed to ...
3,0,my whole body feels itchy and like its on fire,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all....",no it s not behaving at all i m mad why a...
5,0,@Kwesidei not the whole crew,not the whole crew
6,0,Need a hug,Need a hug
7,0,@LOLTrish hey long time no see! Yes.. Rains a...,hey long time no see Yes Rains a bit on...
8,0,@Tatiana_K nope they didn't have it,nope they didn t have it
9,0,@twittera que me muera ?,que me muera


In [5]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
df["text_cleaned"]=df["text_cleaned"].str.lower().apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

df.head(10)


,sentiment,text,text_cleaned
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",awww bummer shoulda got david carr third day
1,0,is upset that he can't update his Facebook by ...,upset update facebook texting might cry result...
2,0,@Kenichan I dived many times for the ball. Man...,dived many times ball managed save rest go bounds
3,0,my whole body feels itchy and like its on fire,whole body feels itchy like fire
4,0,"@nationwideclass no, it's not behaving at all....",behaving mad see
5,0,@Kwesidei not the whole crew,whole crew
6,0,Need a hug,need hug
7,0,@LOLTrish hey long time no see! Yes.. Rains a...,hey long time see yes rains bit bit lol fine t...
8,0,@Tatiana_K nope they didn't have it,nope
9,0,@twittera que me muera ?,que muera


In [6]:
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()

tokenized_tweet = df['text_cleaned'].apply(lambda x: tknzr.tokenize(x))

tokenized_tweet.head(10)

0    [awww, bummer, shoulda, got, david, carr, thir...
1    [upset, update, facebook, texting, might, cry,...
2    [dived, many, times, ball, managed, save, rest...
3              [whole, body, feels, itchy, like, fire]
4                                 [behaving, mad, see]
5                                        [whole, crew]
6                                          [need, hug]
7    [hey, long, time, see, yes, rains, bit, bit, l...
8                                               [nope]
9                                         [que, muera]
Name: text_cleaned, dtype: object

In [7]:
from nltk import PorterStemmer

ps = PorterStemmer()

tokenized_tweet = tokenized_tweet.apply(lambda x: [ps.stem(i) for i in x])

tokenized_tweet.head()


0    [awww, bummer, shoulda, got, david, carr, thir...
1    [upset, updat, facebook, text, might, cri, res...
2    [dive, mani, time, ball, manag, save, rest, go...
3               [whole, bodi, feel, itchi, like, fire]
4                                    [behav, mad, see]
Name: text_cleaned, dtype: object

In [10]:
for i in range(len(tokenized_tweet)):
    tokenized_tweet[i] = ' '.join(tokenized_tweet[i])

df['text_cleaned'] = tokenized_tweet
df.head()


,sentiment,text,text_cleaned
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",awww bummer shoulda got david carr third day
1,0,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,@Kenichan I dived many times for the ball. Man...,dive mani time ball manag save rest go bound
3,0,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,"@nationwideclass no, it's not behaving at all....",behav mad see


In [11]:

# change sentiment score to be 0 as negative and 1 as positive
df['sentiment'].replace(to_replace=4, value=1, inplace=True)

# shuffle the data (the original data is sorted by the sentiments, so that we cannot choose the first N rows to train)
df_shuffled=df.sample(frac=1).reset_index(drop=True)

df.to_csv('data_cleaned.csv', index=False)
